In [1]:
from bs4 import BeautifulSoup
import json
import re
import requests
import datetime
import pandas as pd
from urllib.parse import urlparse

In [23]:
import sys
from pathlib import Path
sys.path.append(Path().cwd().parent.as_posix())

from hospwork import get_work_page
from hospwork import findjobtype
from hospwork.tool.job import clean_unused_str
from hospwork.tool.time import clean_date

In [25]:
name = '國立臺灣大學醫學院附設醫院'
url_base='https://www.ntuh.gov.tw/ntuh'
url_work='/Recruit.action'
url_ajax = '/RecruitAjax.action'
url_full=url_base+url_work
headers = { 'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'}
ntuh_req = requests.get(url_full, headers=headers)
my_params = {'jobtype': '-1', 'q_seekValue': '','pagenum':'1'}

In [7]:
ntuh_work_pages=requests.get(url_base+url_ajax, params=my_params, cookies = ntuh_req.cookies)

In [4]:
my_params = {'jobtype': '-1', 'q_seekValue': '','pagenum':'1'}

In [9]:
ntu_work_data=json.loads(ntuh_work_pages.content)

# Get Pages link

In [15]:
totalcount = ntu_work_data['totalPages']
totalPages_work = ntu_work_data['totalPages']

# Get work table

In [36]:
def _get_ntuh_work_table_one_page(url_base, ntu_work_data, work_table):
    for i, item in enumerate(ntu_work_data['queryList']):
        title = clean_unused_str(item['title'],name)
        origantion = item['jobDepno']
        begin_date = item['adate_sh']
        if item['edatestr'] != '':
            #dead_line = clean_date(item['edatestr'], name)
            dead_line = datetime.datetime.strptime(item["odate"],"%Y-%m-%dT%H:%M:%S").date()
        else:
            dead_line = 'please check page'

        if item['recruitNo'] and "分院" not in origantion:
            #link = 'https://reg.ntuh.gov.tw/WebApplication/Administration/NtuhGeneralSelect/Entry.aspx?selectno='+item['recruitNo']
            detail_link = 'https://reg.ntuh.gov.tw/NtuhGeneralSelect/Openpdf.aspx?SelectNo='+item['recruitNo']+'&FileName='+item['recruitNo']+'.pdf'
            resume_link = 'https://reg.ntuh.gov.tw/NtuhGeneralSelect/Entry.aspx?selectno='+item['recruitNo']
        elif "分院" in origantion:
            break
        else:
            detail_link = url_base+item['ctx'].lstrip('<p><a href="../').split('"><span ')[0]
            resume_link = ''
        print('#{}召聘職稱: {} 召聘單位: {}\n 期限: {}\n 連結：{}'.format(i+1, title, origantion, dead_line, detail_link))
        #work_table.append([title, origantion, dead_line, detail_link, resume_link])

In [37]:
work_table = []
for _page in range(1,totalPages_work):
    my_params = {'jobtype': '-1', 'q_seekValue': '','pagenum':'{}'.format(str(_page))}
    g=requests.get(url_base+url_ajax, params=my_params, cookies = ntuh_req.cookies, headers = headers)
    ntu_work_data=json.loads(g.content)
    _get_ntuh_work_table_one_page(url_base,ntu_work_data,work_table)

#1召聘職稱: 服務員 召聘單位: 護理部
 期限: 2024-08-29
 連結：https://reg.ntuh.gov.tw/NtuhGeneralSelect/Openpdf.aspx?SelectNo=202400276&FileName=202400276.pdf
#2召聘職稱: 助理員 召聘單位: 護理部
 期限: 2024-08-29
 連結：https://reg.ntuh.gov.tw/NtuhGeneralSelect/Openpdf.aspx?SelectNo=202400275&FileName=202400275.pdf
#3召聘職稱: 師(三)級護理師(C) 召聘單位: 護理部、麻醉部以外
 期限: 2024-08-29
 連結：https://www.ntuh.gov.tw/ntuhspan style="font-family:微軟正黑體
#4召聘職稱: 師(三)級護理師(B) 召聘單位: 麻醉部
 期限: 2024-08-29
 連結：https://www.ntuh.gov.tw/ntuhspan style="font-family:微軟正黑體
#5召聘職稱: 師(三)級護理師(A) 召聘單位: 護理部
 期限: 2024-08-29
 連結：https://www.ntuh.gov.tw/ntuhspan style="font-family:微軟正黑體
#6召聘職稱: 師(三)級聽力師 召聘單位: 耳鼻喉部
 期限: 2024-08-29
 連結：https://www.ntuh.gov.tw/ntuhspan style="font-family:微軟正黑體
#7召聘職稱: 師(三)級臨床心理師 召聘單位: 臨床心理中心
 期限: 2024-08-29
 連結：https://www.ntuh.gov.tw/ntuhspan style="font-family:微軟正黑體
#8召聘職稱: 師(三)級醫事放射師(B) 召聘單位: 影像醫學部以外
 期限: 2024-08-29
 連結：https://www.ntuh.gov.tw/ntuhspan style="font-family:微軟正黑體
#9召聘職稱: 師(三)級醫事放射師(A) 召聘單位: 影像醫學部
 期限: 2024-08-29
 連結：https://

In [8]:
def get_each_page_wrok_table(soup,work_table):
    for work in soup.find_all('table',class_="table table-striped table-hover news_table")[0].find("tbody").find_all("tr"):
        all_td = work.find_all("td")
        title = all_td[0].text
        place = '察看連結的簡章'
        deadline = all_td[2].text
        
        link=url_base+all_td[3].find('a').get('href')
        if "錄取名單" in title:
            print('skip: ',title)
        elif "甄試名單" in title:
            print('skip: ',title)
        elif "複試名單" in title:
            print('skip: ',title)
        elif "志願服務工作" in title:
            print('skip: ',title)
        elif "甄審會" in title:
            print('skip: ',title)
        elif "時薪人員" in title:
            print('skip: ',title)
        else:
            originization = re.search("\B院((.*)[室,部,中心])",title).group(1)
            new_title = re.search("\B聘((.*)[員,師,廚])",title).group(1)
            print("召聘職稱",new_title,'院區',place,"截止日期",deadline,"職缺單位",originization,"links",link)
            work_table.append([new_title, deadline, originization, place, link ])
    return work_table

In [9]:
def get_base_web_data(url_base,url_work_table=None):
    if url_work_table != None:
        url = url_base + url_work_table
    else:
        url = url_base
    g=requests.get(url)
    soup=BeautifulSoup(g.content, 'html.parser')
    return soup

In [10]:
work_table = []
for page in pages_link:
    soup = get_base_web_data(page)
    work_table = get_each_page_wrok_table(soup,work_table)

NameError: name 'pages_link' is not defined

In [34]:
ntucc_work_data['queryList'][0]

{'FILEEXT1': None,
 'FILENAME1': None,
 'FILEPATH1': None,
 'FILEREMARK1': None,
 'FILESIZE1': None,
 'SEQ1': None,
 'TITLE1': None,
 'TYPENO1': None,
 'adate': '2022-07-06T00:00:00',
 'adate_ch': '111/07/06',
 'adate_sh': '2022/07/06',
 'attachNum1': 0,
 'attachNum2': 0,
 'attachNum3': 0,
 'attachNum4': 0,
 'attachNum5': 0,
 'attachNum6': 0,
 'createDate': '2022-06-27T15:09:01',
 'createUser': 'HR',
 'depno': 'ntucc',
 'edate': '2022-07-13T00:00:00',
 'edate_ch': '111/07/13',
 'edate_sh': '2022/07/13',
 'id': '2c9a7cf97fde99380181a3fe00f501ff',
 'isuse': '1',
 'item1list': None,
 'jobDepnm': '人事室',
 'jobDepno': '19',
 'jobtype': '3',
 'lang': '1',
 'mid': 4,
 'modifyDate': '2022-07-06T13:25:09',
 'modifyUser': 'HR',
 'odate': '2022-10-31T00:00:00',
 'recruitFile': [{'FILEID_A': None,
   'FILE_A': None,
   'FILE_AContentType': None,
   'FILE_AFileName': None,
   'ID_A': None,
   'REMARK_A': None,
   'SEQ_A': None,
   'TYPE_A': None,
   'createDate': '2022-06-27T15:31:24',
   'createUse

In [110]:
re.search("\B聘((.*)師)","公告本院護理部公開甄選院聘護理師(手術室)儲備").group(1)

'護理師'

In [11]:
work_table=pd.DataFrame(work_table, columns=['召聘職稱','期限' ,"職缺單位" , '院區','連結'])

print(work_table)

Empty DataFrame
Columns: [召聘職稱, 期限, 職缺單位, 院區, 連結]
Index: []


In [ ]:
https://www.ntucc.gov.tw/ntucc/RecruitAjax!download.action?id=2c9a7cfa7fde98350181c6e991d60268

In [5]:
clean_date("202024/08/30","")

ValueError: time data '202024-08-30' does not match format '%Y-%m-%d'

In [35]:
import datetime
datetime.datetime.strptime("2024-08-29T00:00:00","%Y-%m-%dT%H:%M:%S")

datetime.datetime(2024, 8, 29, 0, 0)